<a href="https://colab.research.google.com/github/praveensrikar/DiffusionLM/blob/main/CosineLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

## Install right packages for Cosine diffusion LM
```scrpyt
!pip install -q --upgrade pip
!pip install -q huggingface_hub==0.28.1 transformers==4.41.1 wandb
!pip install -q spacy==3.8.2 numpy==2.0.0 datasets
!pip install mpi4py
!pip install pytorch torchvision torchaudio cudatoolkit
```

In [1]:
!pip install -q --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.2 MB/s eta 0:00:00


In [2]:
!pip install -q huggingface_hub==0.28.1 transformers==4.41.1 wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [transformers]


In [3]:
!pip install -q spacy==3.8.2 numpy==2.0.0 datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [spacy]


In [4]:
!pip install mpi4py

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.3-cp311-cp311-linux_x86_64.whl size=4442049 sha256=9885b3c212418e033c5652696e5a7518cc7a2d75331755457c0034ecd0cecc16
  Stored in directory: /root/.cache/pip/wheels/5c/56/17/bf6ba37aa971a191a8b9eaa188bf5ec855b8911c1c56fb1f84
Successfully built mpi4py


In [5]:
#!pip install pytorch torchvision torchaudio cudatoolkit

## Setup Drive

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
BASE_DIR= "/content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/CosineDiffusionLLM"
%cd {BASE_DIR}

/content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/CosineDiffusionLLM


## Import Standard libraries

```python
import torch datasets spacy wandb torch sys
import numpy as np
```

In [8]:
import sys
import os
import shutil

import numpy as np

import datasets
import spacy
import wandb
import torch

In [9]:
if torch.cuda.is_available():
  print(f"GPU  available \n Device name :'{torch.cuda.get_device_name(0)}'")
else:
  print("No GPU available. Exiting the execution...")
  sys.exit()

GPU  available 
 Device name :'NVIDIA L4'


## Cloning the Base paper Github code



In [10]:
# Define the name of the folder to check and potentially delete
folder_to_check = "DiffusionLM"

# Check if the folder exists and is a directory
if os.path.exists(folder_to_check) and os.path.isdir(folder_to_check):
    print(f"Folder '{folder_to_check}' found. Deleting...")
    # Delete the folder and its contents
    shutil.rmtree(folder_to_check)
    print(f"Folder '{folder_to_check}' deleted.")
else:
    print(f"Folder '{folder_to_check}' not found. Cloning...")

# Clone the Diffusion-LM repository from GitHub
!git clone https://github.com/XiangLi1999/Diffusion-LM.git

# Define the old and new folder names for renaming
old_folder_name = "Diffusion-LM"
new_folder_name = "DiffusionLM"

# Attempt to rename the cloned directory
try:
  os.rename(old_folder_name, new_folder_name)
  print(f"Directory '{old_folder_name}' has been renamed to '{new_folder_name}'.")
except OSError as e:
    # Handle potential errors during renaming
    print(f"Error renaming directory: {e}")

Folder 'DiffusionLM' found. Deleting...
Folder 'DiffusionLM' deleted.
Cloning into 'Diffusion-LM'...
remote: Enumerating objects: 2173, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 2173 (delta 24), reused 11 (delta 11), pack-reused 2108 (from 1)
Receiving objects: 100% (2173/2173), 46.02 MiB | 15.28 MiB/s, done.
Resolving deltas: 100% (555/555), done.
Updating files: 100% (1904/1904), done.
Directory 'Diffusion-LM' has been renamed to 'DiffusionLM'.


## Downloading E2E database

In [11]:
# Change the current working directory to the renamed folder within the base directory
%cd {BASE_DIR}/{new_folder_name}

/content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/CosineDiffusionLLM/DiffusionLM


In [12]:
!mkdir -p /content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/CosineDiffusionLLM/DiffusionLM/data/e2e_data
%cd /content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/CosineDiffusionLLM/DiffusionLM/data/e2e_data

print("Downloading E2E dataset...")
!wget https://raw.githubusercontent.com/tuetschek/e2e-dataset/master/trainset.csv
!wget https://raw.githubusercontent.com/tuetschek/e2e-dataset/master/devset.csv
!wget https://raw.githubusercontent.com/tuetschek/e2e-dataset/master/testset.csv
print("Download complete.")

# Change back to the main project directory
%cd /content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/CosineDiffusionLLM/DiffusionLM

/content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/CosineDiffusionLLM/DiffusionLM/data/e2e_data
--2025-06-21 15:34:54--  https://raw.githubusercontent.com/tuetschek/e2e-dataset/master/trainset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9340189 (8.9M) [text/plain]
Saving to: ‘trainset.csv’

trainset.csv        100%[===================>]   8.91M  --.-KB/s    in 0.09s   

2025-06-21 15:34:58 (95.4 MB/s) - ‘trainset.csv’ saved [9340189/9340189]

--2025-06-21 15:34:58--  https://raw.githubusercontent.com/tuetschek/e2e-dataset/master/devset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.19

## Installing project libraries

In [13]:
!pwd
!pip install -e improved-diffusion/
# Uninstall the editable installation of transformers
!pip uninstall -y transformers
# Install a compatible version of transformers from PyPI
!pip install transformers==4.41.1

/content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/CosineDiffusionLLM/DiffusionLM
Obtaining file:///content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/CosineDiffusionLLM/DiffusionLM/improved-diffusion
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 136.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 132.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 73.2 MB/s eta 0:00:0

# Preprocess the Dataset

```python
!python /content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/CosineDiffusionLLM/DiffusionLM/improved-diffusion/scripts/preprocess_data.py \
    --data_dir data/e2e_data \
    --dataset datasets/e2e_data \
    --tokenizer gpt2 \
    --output_dir datasets/processed_e2e
```
not required. as we are using E2E not ROC




# Train the Model
Baseline

In [14]:
# !python scripts/run_train.py \
#     --diff_steps 2000 \
#     --model_arch transformer \
#     --lr 0.0001 \
#     --lr_anneal_steps 400000 \
#     --seed 102 \
#     --noise_schedule linear \
#     --in_channel 16 \
#     --modality e2e \
#     --submit no \
#     --padding_mode block \
#     --app "--predict_xstart True --training_mode e2e-simple --vocab_size 821 --e2e_train datasets/processed_e2e/train.json" \
#     --notes "e2e-baseline-linear"h

In [16]:
%cd {BASE_DIR}/{new_folder_name}/improved-diffusion
!python scripts/run_train.py \
--modality e2e \
--bsz 32 \
--lr 1e-04 \
--diff_steps 2000 \
--noise_schedule cosine \
--in_channel 128 \
--hidden_size 128 \
--num_res_blocks 2 \
--dropout 0.1 \
--seed 101 \
--notes "e2e_cosine_colab_t10" \
--submit "no"

/content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/CosineDiffusionLLM/DiffusionLM/improved-diffusion
 OPENAI_LOGDIR=diffusion_models/diff_e2e_block_rand128_conv-unet_lr0.0001_0.0_2000_cosine_Lsimple_h128_s2_d0.1_sd101_e2e_cosine_colab_t10  TOKENIZERS_PARALLELISM=false python scripts/train.py   --checkpoint_path diffusion_models/diff_e2e_block_rand128_conv-unet_lr0.0001_0.0_2000_cosine_Lsimple_h128_s2_d0.1_sd101_e2e_cosine_colab_t10 --model_arch conv-unet --modality e2e --save_interval 50000 --lr 0.0001 --batch_size 32  --diffusion_steps 2000 --noise_schedule cosine  --use_kl False --learn_sigma False  --image_size 8 --num_channels 128 --seed 101 --dropout 0.1 --in_channel 128 --out_channel 128 --padding_mode block --experiment random  --lr_anneal_steps 400000 --weight_decay 0.0 --num_res_blocks 2  
2025-06-21 15:38:31.832854: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point roun

#Evaluation
```python
!python /content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/CosineDiffusionLLM/DiffusionLM/improved-diffusion/scripts/batch_decode.py \
    --model_path /content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/CosineDiffusionLLM/DiffusionLM/checkpoints/e2e_cosine_schedule_baseline/ema_0.9999_200000.pt \
    --out_dir /content/drive/MyDrive/Colab_Code/UA_MS_IS_ML_23-25/Capstone/05_Code/CosineDiffusionLLM/DiffusionLM/generated_text/baseline \
    --batch_size 64
    ```
  